In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from mlnp.device import set_default_continuous_type
import numpy as np

set_default_continuous_type(np.float32)

In [3]:
import numpy as np
import seaborn as sbs
import os
import pickle

from mlnp.dataset.mnist import load as load_mnist, init as init_mnist, download_mnist

np.random.seed(0)

MNIST_PATH = "../data/mnist"

if not os.path.exists(MNIST_PATH):
    os.mkdir(MNIST_PATH)
    init_mnist(MNIST_PATH)

x_train, y_train_idx, x_test, y_test_idx = load_mnist(MNIST_PATH)
# x_train = np.pad(x_train, ((0, 0), (0, 0), (2, 2), (2, 2)))
x_train = ((x_train / 256) - 0.5) / 0.5  # Normalize
y_train = np.zeros((x_train.shape[0], 10), dtype=np.float32)
y_train[np.arange(x_train.shape[0]), y_train_idx] = 1.
# x_test = np.pad(x_test, ((0, 0), (0, 0), (2, 2), (2, 2)))
x_test = ((x_test / 256) - 0.5) / 0.5  # Normalize
y_test = np.zeros((x_test.shape[0], 10), dtype=np.float32)
y_test[np.arange(x_test.shape[0]), y_test_idx] = 1.

In [4]:
from mlnp.nn.base import NeuralNetwork
from mlnp.nn.loss import BinaryCrossEntropyLoss, MSELoss
from mlnp.nn.layer import Softmax, LeakyReLU, Dropout, Sigmoid, Linear, Conv2d, Reshape, Flatten, MaxPool2d
from mlnp.nn.optim import Adam, SGD
import json

nn = NeuralNetwork([
    Flatten(),
    Linear(28 * 28, 512),
    LeakyReLU(0.2),
    Linear(512, 10),
    LeakyReLU(0.2),
    Softmax()
], BinaryCrossEntropyLoss(), SGD(1e-4))

In [5]:
nn._layers[1].mat.dtype

dtype('float32')

In [42]:
from typing import Tuple
import time

def train_nn(batch_size: int, epoch_size: int, save_path: str = "../model.pkl") -> Tuple[np.ndarray, np.ndarray]:
    data_size = len(x_train)
    data_pos = np.arange(len(x_train))

    loss_data = np.zeros(len(range(0, data_size, batch_size)) * epoch_size)
    loss_data_on = 0

    loss_epoch = np.zeros(epoch_size)

    nn.train()

    for epoch in range(1, epoch_size + 1):
        np.random.shuffle(data_pos)
        loss_total = 0.
        for i in range(0, data_size, batch_size):
            p = data_pos[i:min(i + batch_size, data_size)]
            nn.zero_grad()
            result = nn(x_train[p])
            loss = nn.loss(y_train[p])
            nn.backward()
            nn.step()
            loss_total += loss.item()
            print(f"\r{epoch}; {i}/{data_size}; {round(loss.item(), 5)} {' ' * 30}", end="  ")

            loss_data[loss_data_on] = loss
            loss_data_on += 1

        loss_total /= len(range(0, data_size, batch_size))
        print(f"\r{epoch}; Loss total: {round(loss_total, 5)}. {' ' * 30}")

        loss_epoch[epoch - 1] = loss_total

    if save_path is not None:
        with open(save_path, "wb+") as f:
            pickle.dump(nn.state_dict(), f)

    return loss_data, loss_epoch


loss_on_iter, loss_on_epoch = train_nn(4, 10)

1; 3364/60000; 0.10029                                 

D:\Programming\numpy\MLP\src\ml\nn\optim.py:263: RuntimeWarning: divide by zero encountered in divide
  gs = tuple(np.where(np.abs(g) < 1e-3, 0, 1. / g) for g in gs)


1; 14568/60000; 0.02089                                 


KeyboardInterrupt



In [ ]:
sbs.lineplot(y=loss_on_iter, x=np.arange(loss_on_iter.shape[0]) + 1)

In [ ]:
sbs.lineplot(y=loss_on_epoch, x=np.arange(len(loss_on_epoch)) + 1)

In [43]:
from typing import Tuple


def test_nn(batch_size: int = 256, model_path: str = "../model.pkl",
            data_set: Tuple[np.ndarray, np.ndarray] = (x_test, y_test)) -> Tuple[int, int, int, float]:
    # with open(model_path, "rb+") as f:
    #     nn.load_state_dict(pickle.load(f))

    data, lbl = data_set

    total = len(data)
    data_pos = np.arange(total)

    correct = 0
    loss = 0.
    epoch_cnt = len(range(0, total, batch_size))

    cnt = 0

    nn.eval()

    for i in range(0, total, batch_size):
        p = data_pos[i: min(i + batch_size, total)]
        x = data[p]
        y = lbl[p]

        nn.zero_grad()
        result = nn(x)

        loss_round = nn.loss(y)

        loss += loss_round
        correct_round = np.sum(np.argmax(result, axis=1) == np.argmax(y, axis=1))
        correct += correct_round

        cnt += 1
        print(f"\r{cnt}/{epoch_cnt}; Loss: {round(loss_round, 5)}; "
              f"Accuracy: {round(correct_round / x.shape[0], 5)} {' ' * 30}", end="")

    print()
    print(f"Test Result:")
    print(f"Correct: {correct}")
    print(f"Wrong: {total - correct}")
    print(f"Loss: {round(loss / epoch_cnt, 5)}")
    print(f"Accuracy: {round(correct / total, 5)}")

    return total, correct, total - correct, loss


test_nn()

40/40; Loss: 0.010649999603629112; Accuracy: 0.875                                  
Test Result:
Correct: 8292
Wrong: 1708
Loss: 0.02304
Accuracy: 0.8292


(10000, 8292, 1708, 0.9216675735078752)

# Compare to traditional algorithms

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [ ]:
def report(model):
    out, targ = np.argmax(y_test, axis=1), np.argmax(lin_reg.predict(x_test), axis=1)
    report = classification_report(out, targ)
    print(report)
    correct = np.sum(out == targ)
    return out.shape[0], correct, out.shape[0] - correct

## Linear Regression

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)

In [ ]:
total, lin_reg_correct, lin_reg_wrong = report(lin_reg)

## Random Forest

In [ ]:
forest = RandomForestClassifier()
forest.fit(x_train, y_train)

In [ ]:
total, forest_correct, forest_wrong = report(forest)

## Decision Tree

In [ ]:
d_tree = DecisionTreeClassifier()
d_tree.fit(x_train, np.argmax(y_train, axis=1))

In [ ]:
d_tree_result = d_tree.predict(x_test)
d_tree_correct = np.sum(d_tree_result == np.argmax(y_test, axis=1))
d_tree_wrong = total - d_tree_correct
print(classification_report(np.argmax(y_test, axis=1), d_tree_result))